In [71]:
import pandas as pd
df = pd.read_csv("nypd.csv")


### Dropping empty columns values

In [5]:
df.dropna(subset=['LATITUDE'], inplace=True)
df.dropna(subset=['LONGITUDE'], inplace=True)
df.dropna(subset=['CONTRIBUTING FACTOR VEHICLE 1'], inplace=True)
df = df.drop(df[df['CONTRIBUTING FACTOR VEHICLE 1'] =='Unspecified'].index)
inattetion =  df.loc[df['CONTRIBUTING FACTOR VEHICLE 1'] == "Driver Inattention/Distraction"]
failure = df.loc[df['CONTRIBUTING FACTOR VEHICLE 1'] == "Failure to Yield Right-of-Way"]
otherVehicle = df.loc[df['CONTRIBUTING FACTOR VEHICLE 1'] == "Other Vehicular"]
fatigued =  df.loc[df['CONTRIBUTING FACTOR VEHICLE 1'] == "Fatigued/Drowsy"]
inattention25k = inattetion.iloc[-10000:]
otherVehicle25k = otherVehicle.iloc[-10000:]
failure25k = failure.iloc[-10000:]
fatigued25k = fatigued.iloc[-10000:]
minLat, minLon = 40.499186, -74.251609
maxLat, maxLon = 40.977385, -73.648929
minLat =  min(df[df['LATITUDE']>39]['LATITUDE'])
maxLat = max(df[df['LATITUDE']>39]['LATITUDE'])
minLon =  min(df[df['LONGITUDE']>-175]['LONGITUDE'])
maxLon =  max(df[df['LONGITUDE']<-72]['LONGITUDE'])
latIncrease = (maxLat- minLat)/50.0
lonIncrease = (maxLon - minLon)/50.0
import numpy as np
Y = np.arange(minLat,maxLat, latIncrease)
X = np.arange(minLon+0.04, maxLon+0.04, lonIncrease)

### Calculating the knn for Driver Inattention/Distraction, Failure to Yield Right-of-Way,Other Vehicular, Fatigued/Drowsy 

In [28]:
### needed to calculate KNN later, took from the book

from __future__ import division
from collections import Counter
import math

import matplotlib.pyplot as plt
def vector_subtract(v, w):
    """subtracts two vectors componentwise"""
    return [v_i - w_i for v_i, w_i in zip(v,w)]

def dot(v, w):
    """v_1 * w_1 + ... + v_n * w_n"""
    return sum(v_i * w_i for v_i, w_i in zip(v, w))

def sum_of_squares(v):
    """v_1 * v_1 + ... + v_n * v_n"""
    return dot(v, v)

def magnitude(v):
    return math.sqrt(sum_of_squares(v))

def squared_distance(v, w):
    return sum_of_squares(vector_subtract(v, w))

def distance(v, w):
   return math.sqrt(squared_distance(v, w))

def majority_vote(labels):
    """assumes that labels are ordered from nearest to farthest"""
    vote_counts = Counter(labels)
    winner, winner_count = vote_counts.most_common(1)[0]
    num_winners = len([count 
                       for count in vote_counts.values()
                       if count == winner_count])

    if num_winners == 1:
        return winner                     # unique winner, so return it
    else:
        return majority_vote(labels[:-1]) # try again without the farthest


def knn_classify(k, labeled_points, new_point):
    """each labeled point should be a pair (point, label)"""
    
    # order the labeled points from nearest to farthest
    by_distance = sorted(labeled_points,
                         key=lambda (point, _): distance(point, new_point))

    # find the labels for the k closest
    k_nearest_labels = [label for _, label in by_distance[:k]]

    # and let them vote
    return majority_vote(k_nearest_labels)
allCrimes = []
for index,row in inattention25k.iterrows():
    allCrimes.append(([row['LATITUDE'], row['LONGITUDE']],'inattention'))
for index,row in fatigued25k.iterrows():
    allCrimes.append(([row['LATITUDE'], row['LONGITUDE']],'fatigued'))
for index,row in failure25k.iterrows():
    allCrimes.append(([row['LATITUDE'], row['LONGITUDE']],'failure'))
for index,row in otherVehicle25k.iterrows():
    allCrimes.append(([row['LATITUDE'], row['LONGITUDE']],'otherVehicle'))
#creating the csv for knn=5
inattention5 = open("inattention5.csv",'w')
inattention5.write("name,lat,lon\n")
fatigued5 = open("fatigued5.csv",'w')
fatigued5.write("fatigued5,lat,lon\n")
otherVehicle5 = open("otherVehicle5.csv",'w')
otherVehicle5.write("name,lat,lon\n")
failure5 = open("failure5.csv",'w')
failure5.write("name,lat,lon\n")



for y in Y:
    for x in X: 
        c = knn_classify(5, allCrimes, [y,x])
        if c=='inattention':
            inattention5.write('inattention, %f, %f \n' %( y, x))
        if c=='fatigued':
            fatigued5.write('fatigued5, %f, %f \n' %( y, x))
        if c=='otherVehicle':
            otherVehicle5.write('otherVehicle, %f, %f \n' %( y, x))
        if c=='failure':
            failure5.write('failure, %f, %f \n' %( y, x))
            
        
inattention5.close()
fatigued5.close()
otherVehicle5.close()
failure5.close()
#creating the csv for knn=10

inattention10 = open("inattention10.csv",'w')
inattention10.write("name,lat,lon\n")
fatigued10 = open("fatigued10.csv",'w')
fatigued10.write("fatigued10,lat,lon\n")
otherVehicle10 = open("otherVehicle10.csv",'w')
otherVehicle10.write("name,lat,lon\n")
failure10 = open("failure10.csv",'w')
failure10.write("name,lat,lon\n")

for y in Y:
    for x in X: 
        c = knn_classify(10, allCrimes, [y,x])
        if c=='inattention':
            inattention10.write('inattention, %f, %f \n' %( y, x))
        if c=='fatigued':
            fatigued10.write('fatigued10, %f, %f \n' %( y, x))
        if c=='otherVehicle':
            otherVehicle10.write('otherVehicle, %f, %f \n' %( y, x))
        if c=='failure':
            failure10.write('failure, %f, %f \n' %( y, x))
            
        
inattention10.close()
fatigued10.close()
otherVehicle10.close()
failure10.close()

#creating the csv for knn=30
inattention30 = open("inattention30.csv",'w')
inattention30.write("name,lat,lon\n")
fatigued30 = open("fatigued30.csv",'w')
fatigued30.write("fatigued30,lat,lon\n")
otherVehicle30 = open("otherVehicle30.csv",'w')
otherVehicle30.write("name,lat,lon\n")
failure30 = open("failure30.csv",'w')
failure30.write("name,lat,lon\n")

for y in Y:
    for x in X: 
        c = knn_classify(30, allCrimes, [y,x])
        if c=='inattention':
            inattention30.write('inattention, %f, %f \n' %( y, x))
        if c=='fatigued':
            fatigued30.write('fatigued30, %f, %f \n' %( y, x))
        if c=='otherVehicle':
            otherVehicle30.write('otherVehicle, %f, %f \n' %( y, x))
        if c=='failure':
            failure30.write('failure, %f, %f \n' %( y, x))
            
        
inattention30.close()
fatigued30.close()
otherVehicle30.close()
failure30.close()



# Getting the intersections where there were the most crashes by years: 2012,2013,2014,2015

In [407]:
dfCount = df.groupby(['ON STREET NAME','CROSS STREET NAME']).size().reset_index().rename(columns={0:'count'})
failureCount = failure.groupby(['ON STREET NAME','CROSS STREET NAME']).size().reset_index().rename(columns={0:'count'})
fatiguedCount = fatigued.groupby(['ON STREET NAME','CROSS STREET NAME']).size().reset_index().rename(columns={0:'count'})
inattetionCount = inattetion.groupby(['ON STREET NAME','CROSS STREET NAME']).size().reset_index().rename(columns={0:'count'})
otherVehicleCount = otherVehicle.groupby(['ON STREET NAME','CROSS STREET NAME']).size().reset_index().rename(columns={0:'count'})
dfCount = dfCount.sort_values('count', ascending=False)
inattetionCount =  inattetionCount.sort_values('count', ascending=False)
failureCount = failureCount.sort_values('count', ascending=False)
fatiguedCount = fatiguedCount.sort_values('count', ascending=False)
otherVehicleCount = otherVehicleCount.sort_values('count', ascending=False)
#creating masks for each year
mask2012 = ( pd.to_datetime(df['DATE'])  >= '2012-1-1') & ( pd.to_datetime(df['DATE'])  < '2013-1-1')
mask2013 = ( pd.to_datetime(df['DATE'])  >= '2013-1-1') & ( pd.to_datetime(df['DATE'])  < '2014-1-1')
mask2014 = ( pd.to_datetime(df['DATE'])  >= '2014-1-1') & ( pd.to_datetime(df['DATE'])  < '2015-1-1')
mask2015 = ( pd.to_datetime(df['DATE'])  >= '2015-1-1') & ( pd.to_datetime(df['DATE'])  < '2016-1-1')
mask2016 = ( pd.to_datetime(df['DATE'])  >= '2016-1-1') & ( pd.to_datetime(df['DATE'])  < '2017-1-1')

In [111]:
#creating small pandas for each year
df2012 = df.loc[mask2012]
df2013 = df.loc[mask2013]
df2014 = df.loc[mask2014]
df2015 = df.loc[mask2015]
df2016 = df.loc[mask2016]
#getting the intersections for each year and sort them descending 
dfCount2012 = df2012.groupby(['ON STREET NAME','CROSS STREET NAME']).size().reset_index().rename(columns={0:'count'})
dfCount2012 = dfCount2012.sort_values('count', ascending=False)
dfCount2013 = df2013.groupby(['ON STREET NAME','CROSS STREET NAME']).size().reset_index().rename(columns={0:'count'})
dfCount2013 = dfCount2013.sort_values('count', ascending=False)
dfCount2014 = df2014.groupby(['ON STREET NAME','CROSS STREET NAME']).size().reset_index().rename(columns={0:'count'})
dfCount2014 = dfCount2014.sort_values('count', ascending=False)
dfCount2015 = df2015.groupby(['ON STREET NAME','CROSS STREET NAME']).size().reset_index().rename(columns={0:'count'})
dfCount2015 = dfCount2015.sort_values('count', ascending=False)
dfCount2016 = df2016.groupby(['ON STREET NAME','CROSS STREET NAME']).size().reset_index().rename(columns={0:'count'})
dfCount2016 = dfCount2016.sort_values('count', ascending=False)

In [114]:

#creating the csv files with the intersection data
streetNames2012 = []
crossStreetNames2012 = []
count2012 = []
counter =0 
for i in  dfCount2012['ON STREET NAME']:
    streetNames2012.append(i)
    counter = counter +1 
    if counter==10:
        break
counter =0 
for i in  dfCount2012['CROSS STREET NAME']:
    crossStreetNames2012.append(i)
    counter = counter +1 
    if counter==10:
        break      
counter =0 
for i in  dfCount2012['count']:
    count2012.append(i)
    counter = counter +1 
    if counter==10:
        break      
dfs = [df2013,df2014,df2015,df2016]
count2013= []
count2014= []
count2015= []
count2016= []
countAll = [count2013,count2014,count2015,count2016]

x =0 
for dfX in dfs:
    for i in range(0,10):
        aMask = ( dfX['ON STREET NAME']  == streetNames2012[i]) & ( dfX['CROSS STREET NAME']  == crossStreetNames2012[i])
        countAll[x].append( len(dfX.loc[aMask]))
    x = x+1

    
f = open('intersections.csv', 'w')
for i in range(0,10):
    f.write('date,%s With %s,'%(streetNames2012[i],crossStreetNames2012[i]))
f.write('\n')
years =['1-Jan-2013','1-Jan-2014','1-Jan-2015','1-Jan-2016']
year= 0
for i in countAll:
    f.write('%s,'%years[year])
    for j in range(0,10):
        f.write('%i,'%countAll[year][j])
    year = year + 1
    f.write('\n')
f.write('1-Jan-2012,')
for i in count2012: 
     f.write('%i,'%i)
f.write("\n")
f.close()

### Getting the data necessary for day hour heatmap

In [291]:
alltheyears = pd.date_range(start=pd.datetime(2012, 1, 1), periods=365*5, freq='D')
w, h = 7, 24;
Matrix = [[0 for x in range(h)] for y in range(w)] 
weekA_actualResults = []
for i in range(0,365*5):
    k = '{:%m/%d/%Y}'.format(alltheyears[i])
    weekday = alltheyears[i].weekday()
    dfPerDay  = df.loc[df['DATE']==k]
    for j in range(0,24):
        
        if j<9:
            maskH  = (dfPerDay['TIME']>= '%i:00'%j) & (dfPerDay['TIME']<='%i:00'%(j+1))
        else:
            if j==9:
                maskH  = (dfPerDay['TIME']>= '%i:00'%j) & (dfPerDay['TIME']<='%i:00'%(j+1))
            else:
                maskH  = (dfPerDay['TIME']>= '%i:00'%j) & (dfPerDay['TIME']<='%i:00'%(j+1))
        dfPerHour = dfPerDay.loc[maskH]
        Matrix[weekday][j] =  Matrix[weekday][j] + len(dfPerHour)

In [293]:
f = open('data1.tsv', 'w')
f.write("day hour value\n")
for i in range(0,7):
    for j in range(0,24):
            s = '%i %i %i\n'%(i,j, Matrix[i][j])
            f.write(s)
f.close()
MatrixFatigued = [[0 for x in range(h)] for y in range(w)] 
for i in range(0,365*5):
    k = '{:%m/%d/%Y}'.format(alltheyears[i])
    weekday = alltheyears[i].weekday()
    fatiguedPerDay  = fatigued.loc[fatigued['DATE']==k]
    for j in range(0,24):
        if j+1!=24:
            fatiguedPerHour = fatiguedPerDay.between_time( '%i:00:00'%j, '%i:00:00'%((j+1)) )
        else:
            fatiguedPerHour = fatiguedPerDay.between_time( '%i:00:00'%j, '%i:00:00'%(0 ))
        MatrixFatigued[weekday][j] =  MatrixFatigued[weekday][j] + len(fatiguedPerHour)
        
f = open('dataFatigued.tsv', 'w')
f.write("day hour value\n")
for i in range(0,7):
    for j in range(0,24):
        if(j>=2):
            s = '%i %i %i\n'%(i,j, MatrixFatigued[i][j])
            f.write(s)
f.close()
MatrixFailure = [[0 for x in range(h)] for y in range(w)] 
for i in range(0,365*5):
    k = '{:%m/%d/%Y}'.format(alltheyears[i])
    weekday = alltheyears[i].weekday()
    failurePerDay  = failure.loc[failure['DATE']==k]
    for j in range(0,24):
        if j+1!=24:
            failurePerHour = failurePerDay.between_time( '%i:00:00'%j, '%i:00:00'%((j+1)) )
        else:
            failurePerHour = failurePerDay.between_time( '%i:00:00'%j, '%i:00:00'%((0) ))
        MatrixFailure[weekday][j] =  MatrixFailure[weekday][j] + len(failurePerHour)
f = open('dataFailure.tsv', 'w')
f.write("day hour value\n")
for i in range(0,7):
    for j in range(0,24):
        if(j>=2):
            s = '%i %i %i\n'%(i,j, MatrixFailure[i][j])
            f.write(s)
f.close()
MatrixotherVehicle = [[0 for x in range(h)] for y in range(w)] 
for i in range(0,365*5):
    k = '{:%m/%d/%Y}'.format(alltheyears[i])
    weekday = alltheyears[i].weekday()
    otherVehiclePerDay  = otherVehicle.loc[otherVehicle['DATE']==k]
    for j in range(0,24):
        if j+1!=24:
            otherVehiclePerHour = otherVehiclePerDay.between_time( '%i:00:00'%j, '%i:00:00'%((j+1)) )
        else: 
            otherVehiclePerHour = otherVehiclePerDay.between_time( '%i:00:00'%j, '%i:00:00'%((0)) )
        MatrixotherVehicle[weekday][j] =  MatrixotherVehicle[weekday][j] + len(otherVehiclePerHour)
f = open('dataotherVehicle.tsv', 'w')
f.write("day hour value\n")
for i in range(0,7):
    for j in range(0,24):
        if(j>=2):
            s = '%i %i %i\n'%(i,j, MatrixotherVehicle[i][j])
            f.write(s)
f.close()
Matrixinattetion = [[0 for x in range(h)] for y in range(w)] 
for i in range(0,365*5):
    k = '{:%m/%d/%Y}'.format(alltheyears[i])
    weekday = alltheyears[i].weekday()
    inattetionPerDay  = inattetion.loc[inattetion['DATE']==k]
    for j in range(0,24):
        
        if j+1!=24:
            inattetionPerHour = inattetionPerDay.between_time( '%i:00:00'%j, '%i:00:00'%((j+1)) )
        else: 
            inattetionPerHour = inattetionPerDay.between_time( '%i:00:00'%j, '%i:00:00'%((0)) )
        Matrixinattetion[weekday][j] =  Matrixinattetion[weekday][j] + len(inattetionPerHour)
f = open('datainattetion.tsv', 'w')
f.write("day hour value\n")
for i in range(0,7):
    for j in range(0,24):
        if(j>=2):
            s = '%i %i %i\n'%(i,j, Matrixinattetion[i][j])
            f.write(s)
f.close()

### Splitting the data into boroughs and calculating the percentage of the most contributing factors for each borrow, as well the street with the most crashes for each borough

In [296]:
#getting the boroughs
staten = df.loc[df['BOROUGH'] == "STATEN ISLAND"]
bronx = df.loc[df['BOROUGH'] == "BRONX"]
manhattan  =  df.loc[df['BOROUGH'] == "MANHATTAN"]
queens =   df.loc[df['BOROUGH'] == "QUEENS"]
brooklyn = df.loc[df['BOROUGH'] == "BROOKLYN"]

In [309]:
#getting contributings factors for each borrow
statenContributing = pd.value_counts(staten['CONTRIBUTING FACTOR VEHICLE 1'].values, sort=True)
statenContributing = statenContributing[1:8]
bronxContributing = pd.value_counts(bronx['CONTRIBUTING FACTOR VEHICLE 1'].values, sort=True)
bronxContributing = bronxContributing[1:8]
manhattanContributing = pd.value_counts(manhattan['CONTRIBUTING FACTOR VEHICLE 1'].values, sort=True)
manhattanContributing = manhattanContributing[1:8]
queensContributing = pd.value_counts(queens['CONTRIBUTING FACTOR VEHICLE 1'].values, sort=True)
queensContributing = queensContributing[1:8]
brooklynContributing = pd.value_counts(brooklyn['CONTRIBUTING FACTOR VEHICLE 1'].values, sort=True)
brooklynContributing = brooklynContributing[1:8]
#getting the streets for each borough
statenStreets = pd.value_counts(staten['ON STREET NAME'].values, sort=True)
statenStreets = statenStreets[0:8]
bronxStreets = pd.value_counts(bronx['ON STREET NAME'].values, sort=True)
bronxStreets = bronxStreets[1:9]
manhattanStreets = pd.value_counts(manhattan['ON STREET NAME'].values, sort=True)
manhattanStreets = manhattanStreets[0:9]
queensStreets = pd.value_counts(queens['ON STREET NAME'].values, sort=True)
queensStreets = queensStreets[0:9]
brooklynStreets = pd.value_counts(brooklyn['ON STREET NAME'].values, sort=True)
brooklynStreets = brooklynStreets[0:9]

In [352]:
#writting the contributing factors to the csv
from random import randint
f = open("contributing.csv", "w")
f.write("id,value,color\n")
f.write("NewYork,\n")
s= "NewYork.Staten Island"
f.write(s)
f.write(",\n")
for row in statenContributing.iteritems():
    if row[0]!="":
        f.write('%s.%s,%f,%s\n'%(s,row[0],row[1]*100.0/len(staten),'#%06X' % randint(0, 0xFFFFFF)))
        
s= "NewYork.Bronx"
f.write(s)
f.write(",\n")
for row in bronxContributing.iteritems():
    if row[0]!="":
        f.write('%s.%s,%f,%s\n'%(s,row[0],row[1]*100.0/len(bronx),'#%06X' % randint(0, 0xFFFFFF)))

s= "NewYork.Manhattan"
f.write(s)
f.write(",\n")
for row in manhattanContributing.iteritems():
    if row[0]!="":
        f.write('%s.%s,%f,%s\n'%(s,row[0],row[1]*100.0/len(manhattan),'#%06X' % randint(0, 0xFFFFFF)))

s= "NewYork.Queens"
f.write(s)
f.write(",\n")
for row in queensContributing.iteritems():
    if row[0]!="":
        f.write('%s.%s,%f,%s\n'%(s,row[0],row[1]*100.0/len(queens),'#%06X' % randint(0, 0xFFFFFF)))

s= "NewYork.Brooklyn"
f.write(s)
f.write(",\n")
for row in brooklynContributing.iteritems():
    if row[0]!="":
        f.write('%s.%s,%f,%s\n'%(s,row[0],row[1]*100.0/len(brooklyn),'#%06X' % randint(0, 0xFFFFFF)))
f.close()

In [353]:
#writting the street names for each borough on csv
from random import randint
f = open("streets.csv", "w")
f.write("id,value,color\n")
f.write("NewYork,\n")
s= "NewYork.Staten Island"
f.write(s)
f.write(",\n")
for row in statenStreets.iteritems():
    if row[0]!="":
        f.write('%s.%s,%f,%s\n'%(s,row[0],row[1]*100.0/len(staten),'#%06X' % randint(0, 0xFFFFFF)))
        
s= "NewYork.Bronx"
f.write(s)
f.write(",\n")
for row in bronxStreets.iteritems():
    if row[0]!="":
        f.write('%s.%s,%f,%s\n'%(s,row[0],row[1]*100.0/len(bronx),'#%06X' % randint(0, 0xFFFFFF)))

s= "NewYork.Manhattan"
f.write(s)
f.write(",\n")
for row in manhattanStreets.iteritems():
    if row[0]!="":
        f.write('%s.%s,%f,%s\n'%(s,row[0],row[1]*100.0/len(manhattan),'#%06X' % randint(0, 0xFFFFFF)))

s= "NewYork.Queens"
f.write(s)
f.write(",\n")
for row in queensStreets.iteritems():
    if row[0]!="":
        f.write('%s.%s,%f,%s\n'%(s,row[0],row[1]*100.0/len(queens),'#%06X' % randint(0, 0xFFFFFF)))

s= "NewYork.Brooklyn"
f.write(s)
f.write(",\n")
for row in brooklynStreets.iteritems():
    if row[0]!="":
        f.write('%s.%s,%f,%s\n'%(s,row[0],row[1]*100.0/len(brooklyn),'#%06X' % randint(0, 0xFFFFFF)))
f.close()